In [1]:
import tweepy
import json
import pandas as pd
from collections import Counter
from pprint import pprint 
import operator 
import time
from itertools import islice

In [2]:
#PUNTO A

In [3]:
auth = tweepy.OAuthHandler("MwBn5s2sjQ3vrRYMXuc9BtYfw", "36Cc2pA4NN89wTXs3LgUCxSznlpa4v8PoPYw6WTLuH1BF8V5vC")
auth.set_access_token("1037434933835378694-bUg0vb50KLHiaC78YtIa2pxSdnEytU", "VHvbqFILbnDgv2xZesppbs7f1Fk2QPDBIJFB8KoYkUCUH")
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
#api.wait_on_rate_limit=True #en caso que haya problema con el rate limit

In [4]:
trends_mundiales = api.trends_place(1) 


In [5]:
nombre_trends = set([trend['name'] for trend in trends_mundiales[0]['trends']])

In [6]:
top_trends = [] #para uso en el punto siguiente

In [7]:
for i, val in enumerate(islice(nombre_trends, 3)):
    top_trends.append(val)
    print(val)
    

Florida State
Nebraska
#EXO_MusicBankinBerlin


In [8]:
#FIN PUNTO A

In [9]:
#PUNTO B

In [10]:
for t in top_trends:    
    print("########### " + t + " ###########" )
    search_result = api.search(q=t)    
    for i in search_result['statuses']:
        (i['user']['screen_name'])   #para printear los usuarios         

########### Florida State ###########
########### Nebraska ###########
########### #EXO_MusicBankinBerlin ###########


In [11]:
#FIN PUNTO B

In [12]:
#PUNTO C

In [13]:
for t in top_trends:  
    print("########### " + t + " ###########" )
    cnt = Counter() 
    tweets = api.search(q=t)    
    for i in tweets['statuses']:                
        for word in i["text"].split(): 
            cnt[word.lower()] += 1
    sorted_cnt = sorted(cnt.items(),key=operator.itemgetter(1),reverse=True) #python2: iteritems
    pprint(sorted_cnt[:5]) # top 5 palabras 
    

########### Florida State ###########
[('florida', 12), ('rt', 11), ('state', 9), ('to', 5), ('a', 5)]
########### Nebraska ###########
[('nebraska', 12), ('rt', 8), ('troy', 8), ('⚔', 6), ('vs.', 4)]
########### #EXO_MusicBankinBerlin ###########
[('rt', 15),
 ('music', 8),
 ('bank', 8),
 ('#exo_musicbankinberlin', 8),
 ('in', 7)]


In [14]:
#FIN PUNTO C

In [15]:
#PUNTO 2_1_PASIVO

In [16]:
#Usuario -> seguidores -> seguidores a su vez, >10
#herramienta gráficos: d3js
#seguidores (100, 150)
# para agregar python 3 kernel : python3 -m ipykernel install --user

In [17]:
def seguidores_ids(usuario):        
    print(usuario)
    f = []
    seguidores_usuario = [] # top 10 de un usuario
    for page in tweepy.Cursor(api.followers_ids, screen_name=usuario).pages():                         
        f.append(page)        
        time.sleep(60)  

    cant = 1
    for v in f:    
        for vids in v['ids']:
            u = api.get_user(vids) #se obtiene el usuario.            
            if u['followers_count']>10:
                seguidores_usuario.append(u['screen_name'])
                cant+=1
                if cant >20: #del usuario X se toman 20 seguidores
                    break
    return seguidores_usuario

In [18]:
red_usuarios = {} 
search_result = api.search(q=top_trends[1], count=200)    
for i in search_result['statuses']:
    if(i['user']['followers_count'])>10:              
        usuario_X = i['user']['screen_name']                
resul = seguidores_ids(usuario_X)


TheRobert1288


In [19]:
red_usuarios[usuario_X] = resul

for r in resul: # a los 10 del primer usuario, se repite la misma acción
    try:
        red_usuarios[r] = seguidores_ids(r)        
    except tweepy.TweepError:
        pass #ignora el usuario que dio error

ishdira
duncan_mccleo
MDfineart
BoatingHayford
Disappear_Citiz
marlaneto
BeatriceScott54
fancaster
SoccerIdeaFree
DJBoabSpence
NolesJags904
RWltherspoon
chadturnernow
Jesusagpd
Jessicalve3
ProfitingTools
schackpat
KarleeLalia
billbill_meow
gabrielsomma


In [21]:
db_usuarios = {}
list_seguidores = []
for k, v in red_usuarios.items():    
    u = api.get_user(k) #se obtiene el usuario (clave).    
    list_seguidores.append((k,u['statuses_count'])) #numero de tweets del usuario clave
    for i in v: #seguidores de un k usuario
        u = api.get_user(i) #se obtiene el usuario (seguidor).
        list_seguidores.append((i,u['statuses_count'])) #nombre y cantidad de tweets    
    db_usuarios[k]= list_seguidores

TweepError: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)

In [ ]:
json = json.dumps(db_usuarios)
f = open("db_total_usuarios.json","w")
f.write(json)
f.close()